<a href="https://colab.research.google.com/github/eyaler/avatars4all/blob/master/ganozli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo for paper "Liquid Warping GAN: A Unified Framework for Human Motion Imitation, Appearance Transfer and Novel View Synthesis"

### Made just a little bit more accessible by Eyal Gruss (eyalgruss@gmail.com)

##### Original project: https://svip-lab.github.io/project/impersonator

##### Original notebook: https://colab.research.google.com/github/svip-lab/impersonator/blob/master/impersonator.ipynb

##### Impersonator++: https://www.impersonator.org/work/impersonator-plus-plus.html

#### **Stuff I made**:
##### Avatars4all repository: https://github.com/eyaler/avatars4all
##### Notebook for talking head model: https://colab.research.google.com/github/eyaler/avatars4all/blob/master/fomm_bibi.ipynb
##### Notebook for full body models: https://colab.research.google.com/github/eyaler/avatars4all/blob/master/fomm_fufu.ipynb
##### Notebook for live webcam in the browser: https://colab.research.google.com/github/eyaler/avatars4all/blob/master/fomm_live.ipynb
##### Notebook for Wav2Lip audio based lip syncing: https://colab.research.google.com/github/eyaler/avatars4all/blob/master/melaflefon.ipynb
##### List of more generative tools: https://j.mp/generativetools

In [ ]:
#@title Setup
%cd /content
!git clone --depth 1 https://github.com/eyaler/impersonator
!wget -nc -O /content/impersonator/assets/pretrains.zip https://1drv.ws/u/s!AjjUqiJZsj8whLNw4QyntCMsDKQjSg?e=L77Elv
!wget -nc -O /content/impersonator/assets/samples.zip https://1drv.ws/u/s!AjjUqiJZsj8whLNz4BqnSgqrVwAXoQ?e=bC86db
!wget -nc -O /content/impersonator/outputs/checkpoints.zip https://1drv.ws/u/s!AjjUqiJZsj8whLNyoEh67Uu0LlxquA?e=dkOnhQ
!pip install -U youtube-dl
!pip install pillow==7.0.0
!pip install -U imageio
!pip install -U imageio-ffmpeg
!pip install tqdm==4.28.1
!pip install setuptools==39.1.0
!pip install scipy==1.2.1
!pip install torch==1.3.0
!pip install torchvision==0.5.0
!pip install h5py==2.8.0
!pip install scikit_image==0.15.0
!pip install visdom==0.1.8.9
!pip install opencv_contrib_python==3.4.2.17
!pip install matplotlib==3.1.1
!pip install ipdb==0.12.2
!pip install progressbar2==3.46.1
!pip install tensorboardX==1.8
%cd /content/impersonator/thirdparty/neural_renderer
!python setup.py install
!unzip -u /content/impersonator/assets/pretrains.zip -d /content/impersonator/assets
!unzip -u /content/impersonator/assets/samples.zip -d /content/impersonator/assets
!unzip -u /content/impersonator/outputs/checkpoints.zip -d /content/impersonator/outputs
%cd /content/impersonator

In [ ]:
#@title Get the Driver video and Avatar image from the web
#@markdown 1. You can change the URLs to your **own** stuff!
#@markdown 2. Alternatively, you can upload **local** files in the next cells
 
video_url = 'https://www.youtube.com/watch?v=NdEPmitJvaA' #@param {type:"string"}
image_url = 'https://i.pinimg.com/564x/74/9e/96/749e96293399ebe1b8c99d89be522383.jpg' #@param {type:"string"}
 
if video_url:
  !rm -f /content/video.mp4
  !youtube-dl -f 'bestvideo[ext=mp4][height<=360]+bestaudio[ext=m4a]/mp4[height<=360]/mp4' '$video_url' --merge-output-format mp4 -o /content/video
  !mv /content/video.mp4 /content/video 
 
if image_url:
  !wget '$image_url' -O /content/image

In [ ]:
#@title Optionally upload local Driver video { run: "auto" }
manually_upload_video = False #@param {type:"boolean"}
if manually_upload_video:
  from google.colab import files
  import shutil

  %cd /content/sample_data
  try:
    uploaded = files.upload()
  except Exception as e:
    %cd /content
    raise e

  for fn in uploaded:
    shutil.move('/content/sample_data/'+fn, '/content/video')
    break

In [ ]:
#@title Optionally upload local Avatar image { run: "auto" }
manually_upload_image = False #@param {type:"boolean"}
if manually_upload_image:
  from google.colab import files
  import shutil
 
  %cd /content/sample_data
  try:
    uploaded = files.upload()
  except Exception as e:
    %cd /content
    raise e

  for fn in uploaded:
    shutil.move('/content/sample_data/'+fn, '/content/image')
    break

In [ ]:
#@title Optionally shorten Driver video
start_seconds = 0 #@param {type:"number"}
duration_seconds =  60#@param {type:"number"}
start_seconds = max(start_seconds,0)
duration_seconds = max(duration_seconds,0)
 
!mv /content/video /content/full_video
!ffmpeg -i /content/full_video -ss $start_seconds -t $duration_seconds -f mp4 /content/video -y

In [ ]:
#@title Prepare assets
#@markdown If you ran out of RAM this means that the video is too large. You can shorten it above.
 
center_video_to_body = True #@param {type:"boolean"}
crop_video_to_body = False #@param {type:"boolean"}
video_crop_expansion_factor = 1.5 #@param {type:"number"}
center_image_to_body = False #@param {type:"boolean"}
crop_image_to_body = False #@param {type:"boolean"}
image_crop_expansion_factor = 1.05 #@param {type:"number"}
video_crop_expansion_factor = max(video_crop_expansion_factor, 1)
image_crop_expansion_factor = max(image_crop_expansion_factor, 1)
 
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML, clear_output
import cv2
import shutil
import os
import warnings
warnings.filterwarnings("ignore")
 
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
 
def fix_dims(im):
    if im.ndim == 2:
        im = np.tile(im[..., None], [1, 1, 3])
    return im[...,:3]
 
def get_crop(im, center_body=True, crop_body=True, expansion_factor=1, rects=None):
    im = fix_dims(im)
    if (center_body or crop_body) and rects is None:
        rects, _ = hog.detectMultiScale(im, winStride=(4, 4),padding=(8,8), scale=expansion_factor)
    if (center_body or crop_body) and rects is not None and len(rects):
        x0,y0,w,h = sorted(rects, key=lambda x: x[2]*x[3])[-1]
        if crop_body:
            x0 += w//2-h//2
            x1 = x0+h
            y1 = y0+h
        else:
            img_h,img_w = im.shape[:2]
            x0 += (w-img_h)//2
            x1 = x0+img_h
            y0 = 0
            y1 = img_h
    else:
        h,w = im.shape[:2]
        x0 = (w-h)//2
        x1 = (w+h)//2
        y0 = 0
        y1 = h
    return int(x0),int(x1),int(y0),int(y1)
 
def pad_crop_resize(im, x0=None, x1=None, y0=None, y1=None, new_h=256, new_w=256):
    im = fix_dims(im)
    h,w = im.shape[:2]
    if x0 is None:
      x0 = 0
    if x1 is None:
      x1 = w
    if y0 is None:
      y0 = 0
    if y1 is None:
      y1 = h
    if x0<0 or x1>w or y0<0 or y1>h:
        im = np.pad(im, pad_width=[(max(-y0,0),max(y1-h,0)),(max(-x0,0),max(x1-w,0)),(0,0)], mode='edge')
    return resize(im[max(y0,0):y1-min(y0,0),max(x0,0):x1-min(x0,0)], (new_h, new_w))
 
 
source_image = imageio.imread('/content/image')
source_image = pad_crop_resize(source_image, *get_crop(source_image, center_body=center_image_to_body, crop_body=crop_image_to_body, expansion_factor=image_crop_expansion_factor))
imageio.imwrite('/content/crop.jpg', (source_image*255).astype(np.uint8))

shutil.rmtree('/content/images', ignore_errors=True)
os.makedirs('/content/images')
with imageio.get_reader('/content/video', format='mp4') as reader:
  fps = reader.get_meta_data()['fps']
 
  driving_video = []
  rects = None
  try:
      for i,im in enumerate(reader):
          if not crop_video_to_body:
              break
          rects, _ = hog.detectMultiScale(im, winStride=(4, 4),padding=(8,8), scale=video_crop_expansion_factor)
          if rects is not None and len(rects):
              break
      x0,x1,y0,y1 = get_crop(im, center_body=center_video_to_body, crop_body=crop_video_to_body, expansion_factor=video_crop_expansion_factor, rects=rects)
      reader.set_image_index(0)
      for i,im in enumerate(reader):
          driving_video.append(pad_crop_resize(im,x0,x1,y0,y1))
          imageio.imwrite(os.path.join('/content/images','%04d.jpg'%i), (driving_video[-1]*255).astype(np.uint8))
  except RuntimeError:
      pass
 
def vid_display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))
 
    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])
 
    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
 
clear_output()
if rects is not None and len(rects):
    print('first found body in frame %d'%i)
HTML(vid_display(source_image, driving_video).to_html5_video())

In [ ]:
#@title Animate

bg_ks = 7 #@param {type:"slider", min:1, max:25, step:1}
ft_ks = 3 #@param {type:"slider", min:1, max:25, step:1}
only_vis = False #@param {type:"boolean"}
has_detector = True #@param {type:"boolean"}
post_tune = True #@param {type:"boolean"}
front_warp = True #@param {type:"boolean"}
cam_strategy = 'smooth' #@param ['smooth', 'source', 'copy'] {type:"raw"}

only_vis_flag = '--only_vis' if only_vis else ''
has_detector_flag = '--has_detector' if has_detector else ''
post_tune_flag = '--post_tune' if post_tune else ''
front_warp_flag = '--front_warp' if front_warp else ''

%cd /content/impersonator
shutil.rmtree('./outputs/results', ignore_errors=True)
!python run_imitator.py --gpu_ids 0 --model imitator --output_dir ./outputs/results 
    --src_path      /content/crop    \
    --tgt_path      /content/images    \
    --bg_ks $bg_ks   --ft_ks $ft_ks \
    $had_detector_flag $post_tune_flag $front_warp_flag \
    --save_res --cam_strategy $cam_strategy $only_vis_flag
!ffmpeg -f image2 -i /content/impersonator/outputs/results/imitators/gt_%04d.jpg -c:v libx264 -pix_fmt yuv420p out2.mp4 -y
!ffmpeg -f image2 -i /content/impersonator/outputs/results/imitators/pred_%04d.jpg -c:v libx264 -pix_fmt yuv420p /content/output.mp4 -y
#!ffmpeg -i out2.mp4 -filter_complex "[0:v] palettegen" palette.png -y
#!ffmpeg -i out2.mp4 -i palette.png -filter_complex "[0:v][1:v] paletteuse" endresult.gif -y
#!ffmpeg -i /content/output.mp4 -filter_complex "[0:v] palettegen" palette2.png -y
#!ffmpeg -i /content/output.mp4 -i palette2.png -filter_complex "[0:v][1:v] paletteuse" endresult2.gif -y
#!ffmpeg -i endresult.gif -i endresult2.gif -filter_complex hstack outputstack2.gif -y
#!ffmpeg -i outputstack2.gif -f mp4 -c:v libx264 -pix_fmt yuv420p /content/combined.mp4
!ffmpeg -i out2.mp4 -i /content/output.mp4 -filter_complex hstack=inputs=2 /content/combined.mp4 -y
#!ffmpeg -i /content/combined.mp4 -i /content/video -c copy -map 0:v -map 1:a /content/final.mp4 -y

palette="./palette.png"
filters="fps=15,scale=256:-1:flags=lanczos"
!ffmpeg -v warning -i /content/output.mp4 -vf "$filters,palettegen=stats_mode=diff" -y $palette
!ffmpeg -v warning -i /content/output.mp4 -i $palette -lavfi "$filters [x]; [x][1:v] paletteuse=dither=bayer:bayer_scale=5:diff_mode=rectangle" -y /content/output.gif
!ffmpeg -v warning -i /content/output.mp4 -i $palette -lavfi "$filters [x]; [x][1:v] paletteuse" -y /content/output1.gif
!ffmpeg -v warning -i /content/combined.mp4 -vf "$filters,palettegen=stats_mode=diff" -y $palette
!ffmpeg -v warning -i /content/combined.mp4 -i $palette -lavfi "$filters [x]; [x][1:v] paletteuse=dither=bayer:bayer_scale=5:diff_mode=rectangle" -y /content/combined.gif

from IPython.display import HTML, clear_output
from base64 import b64encode

clear_output()
with open('/content/combined.mp4', 'rb') as f:
  data_url = "data:video/mp4;base64," + b64encode(f.read()).decode()
display(HTML("""
<video width=600 controls autoplay loop>
      <source src="%s" type="video/mp4">
</video>""" % data_url))

In [ ]:
#@title Download
#@markdown 1. If it fails try running this cell again.
#@markdown 2. Alternatively, you can manually download "output.mp4", "combined.mp4", "output.gif", "combined.gif" from the folder on the left (click "Refresh" if missing).

print() #see https://github.com/googlecolab/colabtools/issues/468
from google.colab import files
files.download('/content/combined.mp4') #fails for Firefox private window
files.download('/content/output.mp4')
files.download('/content/combined.gif')
files.download('/content/output.gif')